In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib


/Users/Bons/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# Read CSV

lc_df = pd.read_csv("final_df_project_3_0502c.csv")
lc_df.head()

,int_rate,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,earliest_cr_line,tax_liens,pub_rec_bankruptcies,term,loan_amnt,credit_start_date
0,1,68000.0,15.74,0.0,750.0,0.0,0.0,0.0,11.0,0.0,14471.0,30.6,15.0,Individual,Mar-10,0.0,0.0,36,13500.0,3
1,3,110000.0,23.44,0.0,675.0,0.0,39.0,0.0,12.0,0.0,13362.0,50.8,29.0,Individual,Sep-03,0.0,0.0,36,20000.0,1
2,2,27840.0,15.99,0.0,685.0,1.0,0.0,0.0,14.0,0.0,5771.0,30.9,20.0,Individual,Nov-11,0.0,0.0,36,1600.0,3
3,2,28000.0,8.62,0.0,700.0,0.0,49.0,0.0,7.0,0.0,4132.0,18.2,17.0,Individual,Jan-10,0.0,0.0,36,6400.0,3
4,2,72500.0,29.58,0.0,680.0,0.0,47.0,0.0,9.0,0.0,27733.0,74.2,20.0,Individual,Jul-07,0.0,0.0,60,16000.0,2


In [3]:
print(lc_df).shape()

         int_rate  annual_inc    dti  delinq_2yrs  fico_range_low  \
0               1     68000.0  15.74          0.0           750.0   
1               3    110000.0  23.44          0.0           675.0   
2               2     27840.0  15.99          0.0           685.0   
3               2     28000.0   8.62          0.0           700.0   
4               2     72500.0  29.58          0.0           680.0   
...           ...         ...    ...          ...             ...   
1273619         4    107000.0  11.65          3.0           670.0   
1273620         1     65000.0  19.55          1.0           725.0   
1273621         3     37000.0  20.56          0.0           705.0   
1273622         2     41000.0  19.99          1.0           670.0   
1273623         4    105700.0  27.26          1.0           695.0   

         inq_last_6mths  mths_since_last_delinq  mths_since_last_record  \
0                   0.0                     0.0                     0.0   
1                   0

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(lc_df.drop("int_rate", axis=1), figsize=(10,5))
plt.show()

In [4]:
# select features which will be used as X values
selected_features = lc_df[["annual_inc", "dti", "fico_range_low", "pub_rec_bankruptcies", "term", "loan_amnt"]]
selected_features

,annual_inc,dti,fico_range_low,pub_rec_bankruptcies,term,loan_amnt
0,68000.0,15.74,750.0,0.0,36,13500.0
1,110000.0,23.44,675.0,0.0,36,20000.0
2,27840.0,15.99,685.0,0.0,36,1600.0
3,28000.0,8.62,700.0,0.0,36,6400.0
4,72500.0,29.58,680.0,0.0,60,16000.0
...,...,...,...,...,...,...
1273619,107000.0,11.65,670.0,1.0,60,24000.0
1273620,65000.0,19.55,725.0,0.0,36,10000.0
1273621,37000.0,20.56,705.0,0.0,36,10050.0
1273622,41000.0,19.99,670.0,0.0,36,6000.0


In [5]:
# declare X and y
X = selected_features
y = lc_df["int_rate"]

In [6]:
# create a train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
X_train.head()

,annual_inc,dti,fico_range_low,pub_rec_bankruptcies,term,loan_amnt
262207,44000.0,27.00,700.0,0.0,36,10000.0
288391,200000.0,3.50,670.0,0.0,60,15000.0
737430,35000.0,17.83,660.0,0.0,36,3125.0
924661,40000.0,29.70,705.0,1.0,36,12000.0
215721,48000.0,36.65,795.0,0.0,60,15150.0


In [8]:
y_train.head()

262207    1
288391    3
737430    1
924661    2
215721    2
Name: int_rate, dtype: int64

In [9]:
# Scale data using StandardScaler assuming our data is linear // don't scale y
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Other step to scale X:
# X_scaler = StandardScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Running KNN Model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)
print('Accuracy of Knn on training', knn.score(X_train_scaled, y_train))
print('Accuracy of Knn on testing', knn.score(X_test_scaled, y_test))

Accuracy of Knn on training 0.599894764967281
Accuracy of Knn on testing 0.4283409787387936


In [11]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

print('Accuracy of logistic regression on training', logreg.score(X_train_scaled, y_train))
print('Accuracy of logistic regression on testing', logreg.score(X_test_scaled, y_test))

/Users/Bons/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Bons/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy of logistic regression on training 0.4609774904843155
Accuracy of logistic regression on testing 0.45914070083940844


In [12]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train_scaled, y_train)

print('Accuracy of Decision tree on training', dt.score(X_train_scaled, y_train))
print('Accuracy of Decision tree on testing', dt.score(X_test_scaled, y_test))

Accuracy of Decision tree on training 0.998126160163501
Accuracy of Decision tree on testing 0.38279926527970765


In [13]:
# Setting max decision tree depth to help avoid overfitting
dt2 = DecisionTreeClassifier(max_depth=3)
dt2.fit(X_train_scaled, y_train)
print('Accuracy of Decision tree on training', dt2.score(X_train_scaled, y_train))
print('Accuracy of Decision tree on testing', dt2.score(X_test_scaled, y_test))

Accuracy of Decision tree on training 0.45344228713929463
Accuracy of Decision tree on testing 0.451317642804119


In [18]:
# Setting max decision tree depth to help avoid overfitting
dt10 = DecisionTreeClassifier(max_depth=10)
dt10.fit(X_train_scaled, y_train)
print('Accuracy of Decision tree on training', dt2.score(X_train_scaled, y_train))
print('Accuracy of Decision tree on testing', dt2.score(X_test_scaled, y_test))

Accuracy of Decision tree on training 0.4831506759417246
Accuracy of Decision tree on testing 0.4767544777966005


In [14]:
# Linear Discriminant Analysis 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scaled,y_train)
print('Accuracy of Linear Discriminant on training', lda.score(X_train_scaled, y_train))
print('Accuracy of Linear Discriminant on testing', lda.score(X_test_scaled, y_test))

Accuracy of Knn on training 0.4660048656554103
Accuracy of Knn on testing 0.46530064525953135


In [15]:
# Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)
print('Accuracy of GNB on training', gnb.score(X_train_scaled, y_train))
print('Accuracy of GNB on testing', gnb.score(X_test_scaled, y_test))

Accuracy of GNB on training 0.4273479834248964
Accuracy of GNB on testing 0.42678969107486153


In [16]:
# Support Vector Machine
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train_scaled, y_train)
print('Accuracy of SVM on training', svm.score(X_train_scaled, y_train))
print('Accuracy of SVM on testing', svm.score(X_test_scaled, y_test))

Accuracy of SVM on training 0.47742251514071965
Accuracy of SVM on testing 0.47559814987532595


In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)
print('Accuracy of RF on training', svm.score(X_train_scaled, y_train))
print('Accuracy of RF on testing', svm.score(X_test_scaled, y_test))

In [ ]:
# # Defining the Model
# from keras.models import Sequential

# model = Sequential()

In [ ]:
# from keras.layers import Dense

# # if we input annual income, loan amount, fico... (others TBD)
# number_inputs = 3
# #  from inputs we will determin annual income, dti, loan amount, fico, interest
# number_hidden_nodes = 4
# model.add(Dense(units=number_hidden_nodes,
#                 activation='relu', input_dim=number_inputs))

In [ ]:
# # we want to determine loan amount - assuming you get the loan and interest rate
# # OR interest rate per two terms: 36 or 60 months
# number_classes = 2
# model.add(Dense(units=number_classes, activation='softmax'))

In [ ]:
import pickle

# Saving model to disk, "wb" = write in binary
pickle.dump(dt3, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load( open('model.pkl','rb'))
# Test an input [["annual_inc", "dti", "fico_range_low", "pub_rec_bankruptcies", "term", "loan_amnt"]]
print(model.predict([[120000]]))

In [ ]:
X_test_scaled[36]

# info inputted as double bracket
# scale the info located in bracket